In [ ]:
#!pip install yfinance

In [ ]:
#!pip install anvil

In [ ]:
#!pip3 install anvil-uplink

In [ ]:
import numpy as np
import pandas as pd 
import yfinance as yf # Yahoo finance 에서 제공하는 데이터에 접근 가능


In [ ]:
import anvil.server
import anvil.mpl_util
global all_data

In [ ]:
anvil.server.connect("BIYPDYG7WDMRMCAFS3STM2VF-7WWLUHLPEZQXNRDL")

In [ ]:
def ts_train_test_normalize(all_data, time_steps, for_periods):
    """
    input: 
        data: dataframe with dates and price data
    output: 
        X_train, y_train: data from 2013/1/1-2018/12/31 
        X_test : data from 2019- 
        sc :     insantiated MinMaxScaler object fit to the training data 
    """
    # create training and test set 
    ts_train = all_data[:'2018'].iloc[:,0:1].values
    ts_test = all_data['2019':].iloc[:,0:1].values 
    ts_train_len = len(ts_train)
    ts_test_len = len(ts_test)
    
    # scale the data 
    from sklearn.preprocessing import MinMaxScaler 
    sc = MinMaxScaler(feature_range=(0,1))
    ts_train_scaled = sc.fit_transform(ts_train)
    
    # create training data of s samples and t time steps 
    X_train = [] 
    y_train = [] 
    for i in range(time_steps, ts_train_len-1):
        X_train.append(ts_train_scaled[i-time_steps:i, 0])
        y_train.append(ts_train_scaled[i:i+for_periods, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    # Reshaping X_train for efficient modelling 
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1 ))
    
    inputs = pd.concat((all_data["Adj Close"][:'2018'], all_data["Adj Close"]['2019':]), axis=0).values
    inputs = inputs[len(inputs)-len(ts_test)-time_steps:]
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    
    # Preparing X_test 
    X_test = [] 
    for i in range(time_steps, ts_test_len + time_steps - for_periods):
        X_test.append(inputs[i-time_steps:i,0])
    
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    return X_train, y_train , X_test, sc 

In [ ]:
def LSTM_model(X_train, y_train, X_test, sc):
    # create a model
    from keras.models import Sequential
    from keras.layers import Dense, SimpleRNN, GRU, LSTM
    from keras.optimizers import SGD
    
    # The LSTM architecture
    my_LSTM_model = Sequential()
    my_LSTM_model.add(LSTM(units = 50, 
                           return_sequences = True, 
                           input_shape = (X_train.shape[1],1), 
                           activation = 'tanh'))
    my_LSTM_model.add(LSTM(units = 50, activation = 'tanh'))
    my_LSTM_model.add(Dense(units=2))
    
    # Compiling 
    my_LSTM_model.compile(optimizer = SGD(lr = 0.01, decay = 1e-7, 
                                         momentum = 0.9, nesterov = False),
                         loss = 'mean_squared_error')
    
    # Fitting to the training set 
    my_LSTM_model.fit(X_train, y_train, epochs = 50, batch_size = 150, verbose = 0)
    
    LSTM_prediction = my_LSTM_model.predict(X_test)
    LSTM_prediction = sc.inverse_transform(LSTM_prediction)
    
    return my_LSTM_model, LSTM_prediction 

In [ ]:

def actual_pred_plot(preds, all_data):
    """
    Plot the actual vs predition
    """
    actual_pred = pd.DataFrame(columns = ['Adj. Close', 'prediction'])
    actual_pred['Adj. Close'] = all_data.loc['2019':,'Adj Close'][0:len(preds)]
    actual_pred['prediction'] = preds[:,0]
    
    from keras.metrics import MeanSquaredError 
    m = MeanSquaredError()
    m.update_state(np.array(actual_pred['Adj. Close']), np.array(actual_pred['prediction']))
    
    return (actual_pred.plot())

In [ ]:
@anvil.server.callable
def predict(name, date):
  A = yf.download( name,
                   start = '2013-01-01',
                   end = date,
                   progress = False)
  all_data = A[['Adj Close', 'Open', 'High','Low',"Close","Volume"]].round(2)
  all_data.head(10)
  print("There are "+str(all_data[:'2018'].shape[0])+" observations in the training data")
  print("There are "+str(all_data['2019':].shape[0])+" observations in the test data")


  X_train, y_train, X_test, sc = ts_train_test_normalize(all_data, 5,2)
  X_train.shape[0], X_train.shape[1]
  # Convert the 3D shape of X_train to a data frame so we can see: 
  X_train_see = pd.DataFrame(np.reshape(X_train, (X_train.shape[0], X_train.shape[1])))
  y_train_see = pd.DataFrame(y_train)
  pd.concat([X_train_see, y_train_see], axis = 1)

  # Convert the 3D shape of X_test to a data frame so we can see: 
  X_test_see = pd.DataFrame(np.reshape(X_test, (X_test.shape[0], X_test.shape[1])))
  pd.DataFrame(X_test_see)
  my_LSTM_model, LSTM_prediction = LSTM_model(X_train, y_train, X_test, sc)
  LSTM_prediction[1:10]
  actual_pred_plot(LSTM_prediction, all_data)

    
  return anvil.mpl_util.plot_image()


In [ ]:
anvil.server.wait_forever()


There are 1472 observations in the training data
There are 972 observations in the test data


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


31/31 [==============================] - 1s 3ms/step
There are 1472 observations in the training data
There are 972 observations in the test data


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


31/31 [==============================] - 1s 3ms/step
